In [15]:
# Proceso empieza con:
#     Limpieza de frases/palabras que no son relevantes en PQRS
#     20 frases mas comunes para 1,2,3 y 4 palabras (total de 80)
#     Luego, si:
#         Esas frases estan compuestas por palabras que contenian a algo eliminado, se elimina (queda un espacio y no se contempla)
#         Esas frases estan siendo repetidas en otras (ejemplo: Mas rapido, mas rapido contestaron. Nos quedamos con la mas larga, "mas rapido contestaron")
#         De esas 80 quedan unas 20

In [9]:
import numpy as np
import pandas as pd
import re
from nltk.tokenize import word_tokenize
import nltk
from nltk.corpus import stopwords
from nltk import tokenize
from nltk.util import ngrams
from collections import Counter    
from nltk.probability import FreqDist
pd.set_option('display.max_colwidth', -1)

<ipython-input-9-e529fc39078c>:11: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [30]:
stop_words = (pd.read_excel('Encuestas_TemasyFrases_stop_words_sin_tilde1.xlsx')) 
stop_words=list(stop_words['stopwords'])

eliminar_frases=['usuario se comunica','usuaria se comunica','confronta exitosa','usuario solicita información',
 'usuaria solicita información','se informa que','se le indica correo','se realiza','se solicita','se genera certificación de pagos',
 'se envía la solicitud al correo','por parte del usuariose envía','se genera certificación de pagos','del proyecto','se informa',
 'se brinda','usuaria','solicita información','se informa que','usuario solicita','usuaria solicita','retroalimentacion','se toma solicitud',
 'solicita','usuaria se comunica para manifestar inconformidad','usuario se comunica','se indica','se informa','cédula','celular','solicitud',
 'cajero:','punto:','correo:','errada:','correc:','radicado:','nombre:','cc:','correo:','telefono:','direccion:','radicado:','os:',
 'ps','usuario solicita','pago realizado','buenas tardes','datos personales','respuesta al correo electronico','correo electronico se indica','usuaria solicita',
 'adjunta pantallazo','al correo sac','se lee articulo','se adjunta','pantallazo de correo','confronta no exitoso','usuario autoriza','buenas tardes',
 'cordial saludo','correo electronico','dias habiles','buenos dias','respuesta al correo','autoriza respuesta','el dia de hoy','tiempo de gestion de','quedo atenta','muchas gracias','articulo de la ley','autoriza tiempo',
 'ley','buen dia','pronta respuesta','cedula','numero de referencia','hasta la fecha','el nombre del titular']


In [11]:
#Importo
Encuestas = pd.read_excel('Encuestas_FrasesyTemas_muestra_textolibre.xlsx')
# Encuestas.reset_index(inplace=True)
Encuestas.rename(columns={'index':'index_inicial'},inplace=True)


In [12]:
Encuestas.rename(columns={'comentario':'Descripción'},inplace=True)

In [13]:
Encuestas['Descripción']=Encuestas['Descripción'].astype(str)
Encuestas['Descripción']=Encuestas['Descripción'].apply(lambda x: x.lower())
Encuestas['Descripción']=Encuestas['Descripción'].apply(lambda x: re.sub('se\s[a-z]{1,10}','',str(x)))
Encuestas['Descripción']=Encuestas['Descripción'].apply(lambda x: re.sub('á','a',str(x)))
Encuestas['Descripción']=Encuestas['Descripción'].apply(lambda x: re.sub('é','e',str(x)))
Encuestas['Descripción']=Encuestas['Descripción'].apply(lambda x: re.sub('í','i',str(x)))
Encuestas['Descripción']=Encuestas['Descripción'].apply(lambda x: re.sub('ó','o',str(x)))
Encuestas['Descripción']=Encuestas['Descripción'].apply(lambda x: re.sub('ú','u',str(x)))
Encuestas['Descripción']=Encuestas['Descripción'].apply(lambda x: re.sub(r"""[\n\t]"""," ",x))
Encuestas['Descripción']=Encuestas['Descripción'].apply(lambda x: re.sub("""[-*\_¡!@#$:).;–,¿?&'_>/]{1,4}""","",x))
Encuestas['Descripción']=Encuestas['Descripción'].apply(lambda x: re.sub(r"""[0-9]""","",x))

In [20]:
for f in eliminar_frases:
    Encuestas['Descripción']=Encuestas['Descripción'].apply(lambda x: re.sub(f,'',str(x)))

In [21]:
pa_token=re.sub("""[,']""",'',str(list(Encuestas['Descripción'])))
tokenized_word=word_tokenize(pa_token)

In [23]:
# for i in range(0,len(tokenized_word_0)):
#     if tokenized_word_0[i]!=tokenized_word[i]:
#         print(tokenized_word_0[i]+' '+tokenized_word[i])       
# # efecto, siweb,sieweb,pqrs .. identificar las entidades con anterioridad

In [24]:
qty_most_common=30
def gen_frases(qty_words_ngram,max_stopwords,df):
    global Top_words
    global qty_most_common
    Top_words=[]
    Encuestas2=pd.DataFrame()
    ongrams=ngrams(tokenized_word,qty_words_ngram)
    ongrams=list(zip(ongrams))
    ongrams=pd.DataFrame(ongrams)
    ongrams['r']=0
    for i in range(0,qty_words_ngram):
        ongrams['val'+str(i)]=ongrams[0].apply(lambda x: x[i])
        ongrams['w'+str(i)]=0
        index_on_val=ongrams[ongrams['val'+str(i)].isin(stop_words)].index
        ongrams.loc[index_on_val,'w'+str(i)]=1
        ongrams['r']=ongrams['r']+ongrams['w'+str(i)]

    ongrams=ongrams[(ongrams['r']<=max_stopwords)&(ongrams['w'+str(qty_words_ngram-1)]==0)] #La ultima condicion es para que no termine con un stopword
    ongrams_=[]
    ongrams.reset_index(inplace=True)

    for i in range(len(ongrams[0])):
        ongrams_.append(re.sub("""[¡!@)#($:.;,¿?&'_]""","",str(ongrams[0][i]))) #Agregar nan

    
    fdist = FreqDist((ongrams_))
    
    df['frases']=''
    for i in range(0,qty_most_common):
        index_frases=df[(df['Descripción'].str.contains(str(fdist.most_common(qty_most_common)[i][0])))&(df['frases']=='')].index
        index_frases_duplicadas=df[(df['Descripción'].str.contains(str(fdist.most_common(qty_most_common)[i][0])))&(df['frases']!='')].index
#Si, veo que esta en el top, pero que no está en las frases df['Descripción'], es porque son 2 palabras que entre medio tenian una frase que 
# se eliminó y queda un espacio grande entre medio que te lo considera dentro del top, pero cuando lo busca no lo encuentra,
# porque no existe tal cual, sino con un espacio de por medio y una frase sin sentido.
        df.loc[index_frases,'frases']=fdist.most_common(qty_most_common)[i][0]
        Top_words.append(fdist.most_common(qty_most_common)[i][0])
        aux2=df.loc[index_frases_duplicadas,:]
        aux2.loc[:,'frases']=fdist.most_common(qty_most_common)[i][0]
        Encuestas2=pd.concat([aux2,Encuestas2])
#     Encuestas2.drop(columns=['level_0'])
    Encuestas2.reset_index(inplace=True)
#     Encuestas2.drop(columns=['level_0'])
    df=pd.concat([df,Encuestas2])
    Encuestas_con_frases=df[df['frases']!='']
#     df['frases']=df['frases'].apply(lambda x: x.upper())
#     Top_words=fdist.most_common(qty_most_common)
    return df


In [25]:
R_frases2=gen_frases(2,0,Encuestas) 
R_frases2=R_frases2[R_frases2['frases']!='']
top_2_palabras=pd.DataFrame(Top_words)

R_frases3=gen_frases(3,1,Encuestas) 
R_frases3=R_frases3[R_frases3['frases']!='']
top_3_palabras=pd.DataFrame(Top_words)

R_frases4=gen_frases(4,2,Encuestas) 
R_frases4=R_frases4[R_frases4['frases']!='']
top_4_palabras=pd.DataFrame(Top_words)


In [26]:
Encuestas3=pd.concat([Encuestas,R_frases2])
Encuestas3=pd.concat([Encuestas3,R_frases3])
Encuestas3=pd.concat([Encuestas3,R_frases4])
Encuestas3.sort_values(by='index',inplace=True)
# Encuestas3.drop(columns=['level_0','index'],inplace=True)
Encuestas3.reset_index(inplace=True)

In [27]:
#PROCESO PARA LIMIPAR FRASES SEMEJANTES DENTRO DE 4 PALABRAS. Repetir este proceso para 3 y 2 palabras

eliminar=[]
index_eliminar_2=[]
index_eliminar_3=[]
index_eliminar_4=[]
top_2_palabras_n=top_2_palabras
top_4_palabras_n=top_4_palabras
top_3_palabras_n=top_3_palabras

for j in range(0,qty_most_common): 
    
    #Pensado para 20 palabras mas comunes de 4 palabras. Compara contra las de 4 y las de 3
    qs=re.findall('[\w]{1,15}',top_4_palabras_n[0][j])
    # 4 palabras solo va a ir contra 3
    for i in range(0,len(qs)):
        for p1 in range(0,qty_most_common):
            try: #Mira si hay 3 palabras segúidas repetidas dentro de los 4grams, si es asi lo elimina
                if(((qs[i]+' '+qs[i+1]+' '+qs[i+2]) in top_4_palabras_n[0][p1])&(p1!=j)):
                    eliminar.append(top_4_palabras_n[0][p1])
                    top_4_palabras_n[0][p1]='-4'
                    index_eliminar_4.append(p1)
            except:
                pass

            
            try: #Mira si hay 2 palabras segúidas repetidas dentro de los 3grams, si es asi lo elimina
                if((qs[i]+' '+qs[i+1]) in top_3_palabras_n[0][p1]):
                    eliminar.append(top_3_palabras_n[0][p1])
                    index_eliminar_3.append(p1)
                    top_3_palabras_n[0][p1]='-4'
            except:
                pass

            try: #Mira si hay 2 palabras segúidas repetidas dentro de los 2grams, si es asi lo elimina
                if((qs[i]+' '+qs[i+1]) in top_2_palabras_n[0][p1]):
                    eliminar.append(top_2_palabras_n[0][p1])
                    index_eliminar_2.append(p1)
                    top_2_palabras_n[0][p1]='-4'
            except:
                pass
            
    #Pensado para 20 palabras mas comunes de 3 palabras, compara contra las de 3 y las de 2
    qs3=re.findall('[\w]{1,15}',top_3_palabras_n[0][j])
    # 3 palabras solo va a ir contra 2
    for i in range(0,len(qs3)):
        for p1 in range(0,qty_most_common):
            try: #Mira si hay 3 palabras segúidas repetidas dentro de los fourgrams, si es asi lo elimina
                if(((qs3[i]+' '+qs3[i+1]) in top_3_palabras_n[0][p1])&(p1!=j)):
                    eliminar.append(top_3_palabras_n[0][p1])
                    index_eliminar_3.append(p1)
                    top_3_palabras_n[0][p1]='-3'
            except:
                pass

            try: #Mira si hay 2 palabras segúidas repetidas dentro de los fourgrams, si es asi lo elimina
                if((qs3[i]+' '+qs3[i+1]) in top_2_palabras_n[0][p1]):
                    eliminar.append(top_2_palabras_n[0][p1])
                    index_eliminar_2.append(p1)
                    top_2_palabras_n[0][p1]='-3'
            except:
                pass
            

#ELIMINA LAS FRASES QUE EL PROCESO ELIGIÓ LIMPIAR
for b in range(0,len(eliminar)):
    index_a_eliminar=Encuestas3[Encuestas3['frases']==eliminar[b]].index
    Encuestas3.loc[index_a_eliminar,'frases']='' 


In [28]:
Encuestas3.drop_duplicates(inplace=True)

In [29]:
Encuestas3['frases'].unique()


array(['', 'un buen servicio', 'mas informacion', 'mas proyectos',
       'mas apoyo', 'nuevos proyectos', 'mas publicidad', 'poder tener',
       'publicidad para los puntos', 'por parte de efecty', 'cada dia',
       'servicios publicos', 'mas rapido', 'sieweb live',
       'el tiempo de respuesta', 'cada punto', 'mas acompañamiento',
       'la prestacion del servicio', 'puntos nuevos',
       'servicio al cliente', 'parte de los logisticos', 'ser mas',
       'ahorro a la mano', 'seria bueno', 'muchas veces',
       'giros internacionales', 'seria muy bueno', 'el punto de servicio',
       'la devolucion del dinero', 'mas convenios', 'excelente servicio',
       'pendientes de los puntos', 'adulto mayor', 'del grupo aval',
       'nequi y daviplata', 'nequi daviplata', 'tener mas',
       'para mejorar el servicio', 'convenio con bancolombia',
       'fondo nacional del ahorro', 'los tiempos de respuesta',
       'los puntos de atencion', 'detalles para los clientes',
       'el se

In [21]:
# Encuestas3['frases'].unique()
Encuestas3[['index_inicial','Descripción','frases']].sort_values(by='index_inicial').head(5)

,index_inicial,Descripción,frases
230,0,no por el momento,
231,1,ninguno,
5578,2,en el tiempo de respuesta para prevalidar las consignaciones un poco me gustaria que el proceso fuera mas agil,
232,2,en el tiempo de respuesta para prevalidar las consignaciones un poco me gustaria que el proceso fuera mas agil,el tiempo de respuesta
233,3,inestabilidad con algunos proyectos al hacer recaudos como por ejemplo claro y directv,


In [21]:
ruta3= "C:/Users/Usuario/Desktop/Respositorio archivos .py/borrador/Frases_encuestas.xlsx"

In [22]:
from pandas import ExcelWriter
from openpyxl import load_workbook

#Ingreso las bases a hojas al archivo
writer = pd.ExcelWriter(ruta3, engine='openpyxl')
Encuestas3[['index_inicial','Descripción','frases']].to_excel(writer, index=True)
writer.save()
writer.close()